In [1]:
from getpass import getpass
import requests
import pandas as pd
import math
from serlo.mysql import db
from IPython.display import display, Markdown

MAILCHIMP_API_KEY = getpass("MAILCHIMP_API_KEY")
dc = MAILCHIMP_API_KEY.split("-")[1]

MAILCHIMP_API_URL = f"https://{dc}.api.mailchimp.com/3.0/lists/a7bb2bbc4f/members"

MAILCHIMP_API_KEY········


In [2]:
# Hier das Datum eintragen, ab dem Subscriptions gesucht werden sollen.

date = "2020-03-01T00:00:00+00:00"

In [3]:
# TODO: Entscheiden, ob die Description hier rausgenommen wird, da es ein anderer Anwendungsfall ist
df = pd.read_sql(
    """
    Select
        user.id as serlo_id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""",
    db,
)

df.fillna(0, inplace=True)

df["date"] = df["date"].map(pd.to_datetime)
inactive_users = df[
    (df["edits"] + df["reviews"] + df["taxonomy"] == 0)
    & (df["description"] != "NULL")
    & (df["description"] != "")
    & (df["date"] < pd.to_datetime("2021-12-01"))
].copy()
inactive_users.sort_values(["date"], inplace=True)
inactive_users

compact_inactive_users = inactive_users[["serlo_id", "username"]]

/tmp/ipykernel_23419/2075254977.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [4]:
subscribers = requests.get(
    MAILCHIMP_API_URL,
    params={
        "apikey": MAILCHIMP_API_KEY,
        "status": "subscribed",
        "fields": "total_items",
    },
)
total_subscribers = subscribers.json()["total_items"]

display(Markdown(f"### Anzahl der Newsletter-Subscriptions: {total_subscribers}"))

### Anzahl der Newsletter-Subscriptions: 8189

In [5]:
subscribers = []

for i in range(math.ceil(total_subscribers / 1000)):
    subscribers.extend(
        requests.get(
            MAILCHIMP_API_URL,
            params={
                "apikey": MAILCHIMP_API_KEY,
                "status": "subscribed",
                "since_timestamp_opt": date,
                "offset": (i * 1000),
                "count": 1000,
            },
        ).json()["members"]
    )

In [7]:
subscribers_not_opened_newsletter = [
    subscriber
    for subscriber in subscribers
    if subscriber["stats"]["avg_open_rate"] == 0
]

subscribers_with_revealing_email_client = [
    subscriber
    for subscriber in subscribers_not_opened_newsletter
    if subscriber["email_address"].split("@")[1] not in ["protonmail.com"]
]

display(
    Markdown(
        f"### Anzahl Subscriber, die keinen Newsletter geöffnet haben: {len(subscribers_with_revealing_email_client)}"
    )
)

### Anzahl Subscriber, die keinen Newsletter geöffnet haben: 3175

In [8]:
merged_data = {}

for subscriber in subscribers_with_revealing_email_client:
    unique_key = subscriber["merge_fields"]["UNAME"]
    merged_data[unique_key] = subscriber

for _, user in compact_inactive_users.iterrows():
    unique_key = user["username"]
    if unique_key in merged_data:
        merged_data[unique_key].update(user)

linked_accounts = [
    value
    for value in merged_data.values()
    if all(key in value for key in ("serlo_id", "id"))
]

df = pd.json_normalize(linked_accounts)
df.rename(columns={"id": "mailchimp_id"}, inplace=True)
number_of_linked_accounts = len(df)

display(
    Markdown(
        f"""### Anzahl Users, die sich bei der Registrierung für den Newsletter subscribed haben: {number_of_linked_accounts}
        {"Anm.: Es ist kaum zu glauben, dass es so wenig ist. Wir überprüfen die Daten nochmal" 
            if number_of_linked_accounts < 100 else ' '
        }"""
    )
)

### Anzahl Users, die sich bei der Registrierung für den Newsletter subscribed haben: 40
        Anm.: Es ist kaum zu glauben, dass es so wenig ist. Wir überprüfen die Daten nochmal

In [ ]:
"""
Das ist Code, den ich zum Testen verwende.

MY_TEST_MAILCHIMP_API_KEY = ""
dc2 = MY_TEST_MAILCHIMP_API_KEY.split('-')[1]
act_url = f"https://{dc2}.api.mailchimp.com/3.0/lists/66709f13c5/members"
activity = requests.get(act_url, params = {"apikey": MY_TEST_MAILCHIMP_API_KEY})
activity.json()
"""

In [ ]:
"""
members = activity.json()["members"]
test = filter(lambda member: (member["stats"]["avg_open_rate"] != 0) & (member["status"] == "subscribed"), members)
list(test)
"""

In [9]:
def is_possible_fake_account(subscriber):
    return subscriber["merge_fields"]["UNAME"] in [
        user["username"] for _, user in compact_inactive_users.iterrows()
    ]


filtered = [account for account in linked_accounts if is_possible_fake_account(account)]

display(
    Markdown(
        f"### Anzahl Subscriber, die keinen Newsletter geöffnet haben und keine Aktivität auf serlo.org haben: {len(filtered)}"
    )
)

# display(Markdown(f"Ids: {[user['id'] for user in filtered]}"))

### Anzahl Subscriber, die keinen Newsletter geöffnet haben und keine Aktivität auf serlo.org haben: 40

In [ ]:
### Das sind alles Subscriber, von denen wir auch den Account auf serlo.org kennen.
### You can check the profile of the users here: serlo.org/{id}
### Die Liste der Ids samt Usernamen, die von Serlo gelöscht werden sollen, bitte an moritz@serlo.org schicken oder,
### falls nicht verfügbar, an den Admin eures Vertrauens ;)
### Falls nur die Subscription gelöscht werden soll, bitte in den nächsten Abschnitt schauen.

In [ ]:
# TODO: Which information is relevant?
not_linked_subscribers = list(
    filter(
        lambda member: not member["merge_fields"]["UNAME"],
        subscribers_with_revealing_email_client,
    )
)
# print(
#     pd.json_normalize(not_linked_subscribers)[
#         ["id", "email_address", "full_name", "timestamp_signup"]
#     ]
# )

In [ ]:
# Fill in here the mailchimp id (important: not the serlo id!) of the subscribers
# you want to remove from the newsletter list like that [id1, id2, id3, ...]
ids_to_remove = []

In [ ]:
def delete_url(subscriber_hash):
    return f"/lists/a7bb2bbc4f/members/{subscriber_hash}/actions/delete-permanent"


# Uncomment if we are supposed to delete accounts automatically
# for subscriber_hash in ids_to_remove:
#     requests.post(delete_url(subscriber_hash), key: MAILCHIMP_API_KEY)